In [4]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import ruptures as rpt
import csv
import time
import statsmodels.api as sm
from joblib import Parallel,delayed

def function_bai_perron(Y, X, max_cp, min_seg_size):
    # Input:
    # Y: a vector for the objective variable
    # X: (# of samples * # of predictors) matrix for predictors. if you consider a regression model with constant, you should include it in X matrix.
    # max_cp: the maximal number of change points that this function will seach
    # min_seg_size: the minimal number of obsevations in each segment.
    
    # output:
    # optimal_number: the optimal number of change points. It minimise the BIC
    # optimal_location: the optimal lcoations for the optimal number of change points
    # RESULT: the obtained solution for the location of change points for each number of change points
    # RSS: the computed residual sum of squared for each number of change points
    # BIC: the computed value of BIC for each number of change points
    # range_of_change_points: the range of change points that this function has searched.
    
    # matrix [Y, X]
    matrix = np.c_[Y, X]
    
    # list for results
    #RSS = []
    BIC = []
    LOCATION = []
    
    # Given the number of change points, the locations of change points are estimated.
    for i in range(max_cp + 1):
        # fitting 
        algo = rpt.Dynp(model="linear", min_size = min_seg_size, jump = 1).fit(matrix)
        
        # locations of change points given the number of change points:
        result = algo.predict(i)
        
        # Residuals Sum of Squared
        cost = rpt.costs.CostLinear().fit(matrix)
        rss = cost.sum_of_costs(result)
        
        # compute BIC
        n_obs = len(Y)
        
        # calculate degree of freedom to compute BIC
        n_coeff = X.shape[1] # the number of coefficients
        n_seg = len(result) # the number of segments that is equivalent to 1 + the number of change points
        degree_of_freedom = n_coeff*n_seg + 1 + n_seg -1# n_coeff*n_seg + 1 is degree of freedom for the regression model, n_seg -1 is the number of change points that can be also degree of freedom
        
        # calculate the log-likelihood
        # When we have change points, we formulate the regression model by indicator function.
        # Hence we can estimate single regression model even though we have multiple change points
        if n_seg == 1:
            log_likelihood = sm.OLS(matrix[0:result[0], 0], matrix[0:result[0], 1:]).fit().llf
        else:
            matrix_for_hadamard_product = np.zeros((n_obs, n_coeff))
            matrix_for_hadamard_product[0:result[0],:] = 1
            matrix_X = np.multiply(X, matrix_for_hadamard_product)
            for j in range(n_seg-1):
                matrix_for_hadamard_product = np.zeros((n_obs, n_coeff))
                matrix_for_hadamard_product[result[j]:result[j+1],:] = 1
                temp = np.multiply(X, matrix_for_hadamard_product)
                matrix_X = np.c_[matrix_X, temp]
            
            log_likelihood = sm.OLS(matrix[:, 0], matrix_X).fit().llf
        
        # BIC = degree of freedom*log(# of samples) - 2 * log lilkelihood
        bic = degree_of_freedom*np.log(n_obs)- 2*(log_likelihood)

        # save the result
        LOCATION.append(result)
        #RSS.append(rss)
        BIC.append(bic)
    
    # best change point
    optimal_number = BIC.index(min(BIC))
    optimal_location = LOCATION[optimal_number]
    #range_of_change_points = list(range(max_cp + 1))

    return(optimal_number, optimal_location)



def generate_five_variate(n,SNR,rho):
    beta1 = np.array([1,1,1,0,1])
    beta2 = np.array([-1,-1,-1,0,-1])
    p = len(beta1)
    Sigma = np.zeros((p,p));
    for i in range(p):
        for j in range(p):
            Sigma[i,j] = rho**(abs(i-j))
    X = np.random.multivariate_normal(mean =np.zeros(p), cov = Sigma, size = n)
    noise1 = np.random.normal(0,np.sqrt(np.var(X[:int(n/2)]@beta1)/SNR), size = int(n/2)) 
    noise2 = np.random.normal(0,np.sqrt(np.var(X[:int(n/2)]@beta2)/SNR), size = int(n/2)) 
    Y1 = X[:int(n/2)]@beta1 + noise1
    Y2 = X[int(n/2):]@beta2 + noise2
    Y = np.r_[Y1,Y2]
    
    return(Y, X)



SNR_list = np.array([6, 3.52, 2.07, 1.22, 0.71])
rho_list = np.array([0, 0.3, 0.7])
n_list = np.array([1000])

header = ['Repitition','Time','rho', 'SNR', 'n', 'The number of CP', 'Location of CP']#, 'beta_hat', 'nonzero count']
f = open('simulation_five_variate_benchmark_BP.csv', 'w')
writer = csv.writer(f)
writer.writerow(header)
f.close()

def to_repeat(rep):
    for n in n_list:
        for SNR in SNR_list:
            for rho in rho_list:
                tik = time.time()
                Y, X = generate_five_variate(n,SNR,rho)
                opt_num, opt_location= function_bai_perron(Y, X, 5, 3) # max_cp = 5, min_seg = 3
                tok = time.time()
                duration = tok - tik
                # Write result
                resultrow = [rep,duration,rho,SNR, n, opt_num, opt_location]
                f = open('simulation_five_variate_benchmark_BP.csv', 'a')
                writer = csv.writer(f)
                writer.writerow(resultrow)
                f.close()
                
Parallel(n_jobs = 8)(delayed(to_repeat)(rep) for rep in range(500)) #Tomo's machine has 8 cores.


# Answer:
# the number of CP = 1
# Location of CP = [500, 1000]





[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,